In [1]:
import gym
import random
random.seed(0)

def main():
    env = gym.make('SpaceInvaders-v4') #, render_mode="human"
    env.seed(0)

    actions = [i for i in range(0, env.action_space.n)]
    print(actions)

    current_state = env.reset()
    episode_reward = 0
    while True:
        action = random.choice(actions)
        current_state, reward, done, _ = env.step(action)
        episode_reward += reward
        if done:
            print('Reward: %s' % episode_reward)
            break


if __name__ == '__main__':
    main()

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/gym/envs/registration.py:506: UserWarning: WARN: The environment SpaceInvaders-v4 is out of date. You should consider upgrading to version `v5` with the environment ID `ALE/SpaceInvaders-v5`.
  f"The environment {path} is out of date. You should consider "
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/gym/utils/seeding.py:139: DeprecationWarning: WARN: Function `hash_seed(seed, max_bytes)` is marked as deprecated and will be removed in the future. 
  "Function `hash_seed(seed, max_bytes)` is marked as deprecated and will be removed in the future. "
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/gym/utils/seeding.py:176: DeprecationWarning: WARN: Function `_bigint_from_bytes(bytes)` is marked as deprecated and will be removed in the future. 
  "Function `_bigint_from_bytes(bytes)` is marked as deprecated and will be removed in the future. "


[0, 1, 2, 3, 4, 5]


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/gym/utils/seeding.py:48: DeprecationWarning: WARN: Function `rng.randint(low, [high, size, dtype])` is marked as deprecated and will be removed in the future. Please use `rng.integers(low, [high, size, dtype])` instead.
  "Function `rng.randint(low, [high, size, dtype])` is marked as deprecated "


Reward: 140.0
